# Inverted pendulum

In [1]:
import numpy as np
import torch as tch

In [2]:
dev = tch.device("cuda:0")

In [3]:
x = tch.arange(10.0).to(dev)
x ** 2

tensor([ 0.,  1.,  4.,  9., 16., 25., 36., 49., 64., 81.], device='cuda:0')

In [51]:
def step(state: tch.Tensor, action: tch.tensor, dt: float) -> tch.Tensor:
    """
    Inverted pendulum simulation step
    `state` - tensor of shape (batch_size, 3) where last dim items are [moment_of_inertia, angle, angular_velocity]
    `action` - tensor of shape (batch_size, 1) containing torque
    """
    gravity = 9.8
    friction = 2e-1

    moment_of_inertia, angle, angular_velocity = state[:, 0], state[:, 1], state[:, 2]
    torque = action[:, 0]
    acceleration = (torque - friction * angular_velocity) / moment_of_inertia - gravity * angle.cos()
    # TODO: Use RK4 instead of Euler method
    return tch.stack([moment_of_inertia, angle + angular_velocity * dt, angular_velocity + acceleration * dt], dim=1)

In [5]:
from ipycanvas import Canvas, hold_canvas

In [6]:
canvas = Canvas(width=640, height=640)

In [42]:
def draw(state: np.ndarray, canvas: Canvas):
    from math import sin, cos, sqrt

    moment_of_inertia, angle, angular_velocity = state

    with hold_canvas():
        canvas.clear()
        canvas.fill_style = "yellow"
        canvas.stroke_style = "black"

        length = 0.75 * min(canvas.width, canvas.height) / 2
        center = np.array([canvas.width / 2, canvas.height / 2])
        position = center + np.array([length * cos(angle), -length * sin(angle)])
        canvas.stroke_line(*center, *position)
        size = 0.1 * length * sqrt(moment_of_inertia)
        canvas.fill_circle(*position, size)
        canvas.stroke_circle(*position, size)

In [52]:
from time import sleep

state = tch.tensor([[1.0, 1.0, 0.0]])
dt = 1 / 24

for i in range(1000):
    draw(state.squeeze(0).numpy(force=True), canvas)
    state = step(state, tch.zeros((1, 1)), dt)
    sleep(dt)

KeyboardInterrupt: 

In [11]:
canvas

Canvas(height=640, width=640)